In [1]:
import pandas as pd
import seaborn as sns
import rdkit
import torch
import vina
import meeko
import pexpect
import pickle
import numpy as np
from scipy.stats import norm
from typing import Optional, Union, List
from bayes_opt import BayesianOptimization
from bayes_opt.util import load_logs
from bayes_opt.domain_reduction import DomainTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.target_space import TargetSpace
import sys
import json
from contextlib import redirect_stdout

from selectivebayes.interfaces import vaeinterface,vinainterface
from selectivebayes.transformers import SequentialDomainReductionTransformer,SimpleDomainReduction

def visualise(molecules,labels, size,row):
    img = rdkit.Chem.Draw.MolsToGridImage([rdkit.Chem.MolFromSmiles(mol) for mol in molecules],subImgSize=(size,size),molsPerRow=row,legends=labels)
    return img

latent_size=56

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
knowndrugs = {"Gilteritinib":r"CCc1nc(C(=O)N)c(Nc2ccc(N3CCC(CC3)N4CCN(C)CC4)c(OC)c2)nc1NC5CCOCC5", 
              "Quizartinib":r"CC(C)(C)c1cc(no1)NC(=O)Nc2ccc(cc2)c3cn4c5ccc(cc5sc4n3)OCCN6CCOCC6", 
              "Crenolanib":r"O(c5cc4ncn(c1nc3c(cc1)cccc3N2CCC(N)CC2)c4cc5)CC6(COC6)C",
              "Tandutinib":r"CC(C)OC1=CC=C(C=C1)NC(=O)N2CCN(CC2)C3=NC=NC4=CC(=C(C=C43)OC)OCCCN5CCCCC5",
              "Sorafenib":r"CNC(=O)c1cc(ccn1)Oc2ccc(cc2)NC(=O)Nc3ccc(c(c3)C(F)(F)F)Cl",
              "Sunitinib":r"CCN(CC)CCNC(=O)c1c(c([nH]c1C)/C=C\2/c3cc(ccc3NC2=O)F)C",
              "Lestaurtinib":r"C[C@@]12[C@](C[C@@H](O1)n3c4ccccc4c5c3c6n2c7ccccc7c6c8c5C(=O)NC8)(CO)O",
              "Midostaurin":r"C[C@@]12[C@@H]([C@@H](C[C@@H](O1)N3C4=CC=CC=C4C5=C6C(=C7C8=CC=CC=C8N2C7=C53)CNC6=O)N(C)C(=O)C9=CC=CC=C9)OC",
              "Ponatinib":r"Cc1ccc(cc1C#Cc2cnc3n2nccc3)C(=O)Nc4ccc(c(c4)C(F)(F)F)CN5CCN(CC5)C"
              }

fps = {x:rdkit.Chem.RDKFingerprint(rdkit.Chem.MolFromSmiles(knowndrugs[x])) for x in knowndrugs}
def simsearch(molecule):
    if molecule=="failed":
        return 0,0
    molfing = rdkit.Chem.RDKFingerprint(rdkit.Chem.MolFromSmiles(molecule))
    highestsim = 0
    bestmolec = ""
    for fp in fps:
        currsim = rdkit.DataStructs.FingerprintSimilarity(fps[fp],molfing)
        if currsim>highestsim:
            highestsim=currsim
            bestmolec = fp
    return highestsim,bestmolec

In [ ]:
pbounds = {f"f{i}": (-3,3) for i in range(56)}
bounds_transformer = SequentialDomainReductionTransformer(prob=0.5)
optimizer = BayesianOptimization(f=np.square,pbounds=pbounds,verbose=0,bounds_transformer=bounds_transformer)
load_logs(optimizer, logs=["./predictions/minwin05.json"])
seenpreds = set()
predplot = []
x=[]

for i,pred in enumerate(optimizer.res):
    if pred["target"] not in seenpreds:
        predplot.append(pred["target"])
        x.append(i)
        seenpreds.add(pred["target"])
#ax=sns.regplot(x=x,y=predplot)
#ax.set(xlabel="Iteration",ylabel="Objective")
pred_df = pd.DataFrame(data={"pred": predplot,"x": x})
#pred_df["rolling"]=pred_df["pred"].rolling(50,min_periods=5 ,center=True).mean()
pred_df["rolling"]=pred_df["pred"].ewm(alpha=0.05).mean()
sns.set(rc={'figure.figsize':(10,7)})
sns.set_style("white")
ax=sns.scatterplot(data=pred_df,x="x",y="pred")
ax=sns.lineplot(data=pred_df,x="x",y="rolling",color="red")
ax.set(xlabel="Iteration",ylabel="Objective")

In [18]:
pred200 = "./predictions/200pred_list_prob03_doubleexhaustforbest.pk1"
pred500new = "./predictions/500pred_list_newdomainreduction.pk1"
pred200new = "./predictions/200pred_list2_newdomainreduction.pk1"

data = pickle.load(open(pred200,"rb"))
seendata = set()
for molecule in data:
    if molecule[0] in seendata:
        molecule[1]=0
    seendata.add(molecule[0])
data = sorted(data,key=lambda x:x[1],reverse=True)
nummolecs = 20
topmolecules = [molecule[0] for molecule in data[:nummolecs]]
topscores = ["%.3f"%round(molecule[1],3) for molecule in data[:nummolecs]]
#visualise(topmolecules,topscores, size=250,row=5)


In [ ]:
data2 = pickle.load(open("./predictions/500pred_list_newdomainreduction.pk1","rb"))
seendata2 = set()
for molecule in data2:
    if molecule[0] in seendata2:
        molecule[1]=0
    seendata2.add(molecule[0])
data2 = sorted(data2,key=lambda x:x[1],reverse=True)
topmolecules2 = [molecule[0] for molecule in data2[:nummolecs]]

molfing=[]
molecs=[]
for molec in topmolecules:
    molecs.append(molec)
    molfing.append(rdkit.Chem.RDKFingerprint(rdkit.Chem.MolFromSmiles(molec)))
molfing2=[]
molecs2=[]
for molec in topmolecules2:
    molecs2.append(molec)
    molfing2.append(rdkit.Chem.RDKFingerprint(rdkit.Chem.MolFromSmiles(molec)))

for j,molec2 in enumerate(molfing2):
    scores = rdkit.Chem.DataStructs.BulkTanimotoSimilarity(molec2, molfing)
    for i, score in enumerate(scores):
        if score >= 0.4:
            target_id = molecs[i]
            print(f"{molecs2[j]}\t{target_id}\t{score:.3f}")

In [29]:
molecules = []
for molec in topmolecules:
    comps = pcp.get_compounds(molec,"smiles", searchtype='fastsimilarity_3d', listkey_count=1)
    print(comps[0].isomeric_smiles)
    molecules.append(comps[0].isomeric_smiles)
visualise([molecules],None,250,5)

IndexError: list index out of range

In [3]:
affins = pd.read_csv("processed_affins2.csv")
affins.head()

,activity_id,assay_id,molregno,standard_value,standard_type,uniprots,mol_ids,SMILES,confidence_scores,IC50,pKX,KDE,loss_scaling
0,31868,154606,252199,4000.0,IC50,P07872,CHEMBL357278,CC1=NC2=C(S1)C=CC(=C2)OC[C@@H](CN3CCN(CC3)CC(=...,8,1,5.397940,0.220249,1.164387
1,31870,154606,253534,17000.0,IC50,P07872,CHEMBL357119,CC1=NC2=C(S1)C=CC(=C2)OC[C@@H](CN3CCN(CC3)CC(=...,8,1,4.769551,0.144174,1.778788
2,31872,154606,253199,180.0,IC50,P07872,CHEMBL152968,CC1=NC2=C(S1)C=CC(=C2)OC[C@@H](CN3CCN(CC3)CC(=...,8,1,6.744727,0.244066,1.050760
3,31874,51352,253199,6000.0,IC50,P05177,CHEMBL152968,CC1=NC2=C(S1)C=CC(=C2)OC[C@@H](CN3CCN(CC3)CC(=...,8,1,5.221849,0.209584,1.223635
4,31875,51895,253199,37000.0,IC50,P08684,CHEMBL152968,CC1=NC2=C(S1)C=CC(=C2)OC[C@@H](CN3CCN(CC3)CC(=...,8,1,4.431798,0.105412,2.432879


In [5]:
targetaffins=affins.loc[affins["uniprots"]=="P36888"]
fps=[rdkit.Chem.RDKFingerprint(rdkit.Chem.MolFromSmiles(x)) for x in targetaffins["SMILES"]]

 ...]

In [13]:
targetaffins=targetaffins.reset_index()
targetaffins

,index,activity_id,assay_id,molregno,standard_value,standard_type,uniprots,mol_ids,SMILES,confidence_scores,IC50,pKX,KDE,loss_scaling
0,127982,866063,158682,205195,128.0,IC50,P36888,CHEMBL330863,CC1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CN=C3N4CCN(CC4...,8,1,6.892790,0.243835,1.051755
1,129018,872564,158682,205521,8790.0,IC50,P36888,CHEMBL126699,CC1CCCCN1CCCOC2=C(C=C3C(=C2)N=CN=C3N4CCN(CC4)C...,8,1,5.056011,0.203909,1.257695
2,130258,879718,158682,205536,1910.0,IC50,P36888,CHEMBL445636,CC1CCN(CC1)CCCOC2=C(C=C3C(=C2)N=CN=C3N4CCN(CC4...,8,1,5.718967,0.225773,1.135897
3,131163,886058,158682,206266,450.0,IC50,P36888,CHEMBL124035,CC(C)OC1=CC=C(C=C1)NC(=O)N2CCN(CC2)C3=NC=NC4=C...,8,1,6.346787,0.249307,1.028669
4,132780,895434,158682,205365,40.0,IC50,P36888,CHEMBL125898,CC(C)OC1=CC=C(C=C1)NC(=O)N2CCN(CC2)C3=NC=NC4=C...,8,1,7.397940,0.237196,1.081194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,1281451,23363342,2139656,2401765,138.0,IC50,P36888,CHEMBL4531334,CC(C)(C1=CC=C(C=C1)C(=O)NC2=CN3C=C(C=CC3=N2)C4...,9,1,6.860121,0.243835,1.051755
3995,1281776,23372749,2141161,2228127,1000.0,IC50,P36888,CHEMBL4088216,CN1C2=CC=CC=C2OC[C@@H](C1=O)N3CCC4=CN(N=C4C3=O...,9,1,6.000000,0.249472,1.027992
3996,1282071,23373158,2141452,2420095,1000.0,IC50,P36888,CHEMBL4549667,CN1C2=CC=CC=C2OC[C@@H](C1=O)N3CCC4=C(N(N=C4C3=...,9,1,6.000000,0.249472,1.027992
3997,1282363,23373519,2141743,2237689,1000.0,IC50,P36888,CHEMBL4097778,CN1C2=CC=CC=C2OC[C@@H](C1=O)N3CCC4=CN(N=C4C3=O...,9,1,6.000000,0.249472,1.027992


In [22]:
randmols = ['Nc1nnn(-c2ccccc2F)c1CS[C@]12C=CC=C[C@@H]1SC=N2', 'COC(=O)Cc1ccc(NS(C)(=O)=O)cc1', 'Cc1cc(F)cc(C([NH3+])([NH3+])CN(C)C)c1', 'CC1(NC(=O)c2cnn(-c3ccccc3)n2)CCCCC1', 'O=C[C@H](NC(=O)c1cccc2cccnc12)Nc1ccccc1', 'Cc1c(N)cnn1C', 'Cc1cc(N)ccc1F', 'CN(CCN1C(=O)c2ccccc2C1=O)c1ccc(C#CC=O)cc1', 'CC1(C(=O)N(C2CC2)[C@]2(C)SC=C3C=CC=C[C@@H]32)CC1', 'C[C@H]([NH2+]C1(C)CC1)c1oncc1-c1ccsc1', 'Cc1csc([C@H](Br)CNc2ccccc2)n1', 'Cc1ccc[nH+]c1NC[C@]1(c2cccc(F)c2F)CCCS1(=O)=O', 'CC1(C)OCOc2ccc([C@@H](O)c3ccncc3)cc21', 'CCN(C)c1cc[nH+]cc1CNC(=O)N1CCC[C@H]2[NH+]=CN(C)[C@@H]21', 'CC[C@H](C)n1cc(C(=O)NOC)cc([NH3+])c1=O', 'CC(C)(N(C(=O)c1ccsc1)c1ccccc1Br)S(C)(=O)=O', 'Cc1cc(N)cc(Oc2ccncc2N)c1C', 'COc1ccc2c(c1)[C@]1(NC(=O)[C@@H](C)n3cnnn3)C=C(C)C=CC1=C2', 'N[C@H](COc1ccc(-c2ccn[nH]2)cc1)c1ccncc1', 'c1cc(C2CC2)cc(C2C[NH+](C3CCCCCC3)C2)c1', 'C[C@H](C[NH3+])C1(NCc2nc3ccccc3[nH]2)CCC1', 'Nc1n[nH]c(CN[C@]23C=CC(Br)=C[C@H]2CSC3)n1', 'Cc1nc(OC(=O)[C@@H]2CCCN(C)C2)n[nH]1', 'CCOC(=O)[C@@H](F)[C@@H]1CN(Cc2ccccc2)CCN1S(=O)(=O)c1ccccc1', 'O=C(CN1CCc2ccccc2C1)NC1=CCCc2ccccc21', 'C/C=C\\OC(=O)Cc1cn[nH]c(=O)c1N', 'C[C@@H](NC(=O)/C=C\\Cl)c1ccccc1', 'CC(=O)N(c1cccc(C#N)c1)[C@@H]1CCC[C@@]1(C)O', 'COC(=O)Cn1ncnc1COC(C)=O', 'CC(C)=C=O', 'C=CCn1cc(S(=O)(=O)CC(=O)N2CCC[C@H]2CC)cn1', 'CN(c1ncc(C=O)c(-c2ccoc2)n1)S(=O)(=O)/C=C\\c1ccccc1F', 'CN(C)C(=O)[C@@]1(C)CN(C[C@]2(C(=O)Nc3ccccc3F)CCO2)CCO1', 'O=c1c(N[C@@H]2C[NH2+]C[C@@H]2c2ccccc2)coc2c1COC2', 'CCc1coc2c(C)ccc(O)c12', 'C[C@H](S)C(=O)N[C@@H](C[NH3+])c1nccs1', 'CC[C@@H]([NH2+]Cc1cccc2c1COCCO2)c1cncn1C1CCCC1', 'COc1cc(N[C@]2(C)NCC(=O)N2C(=O)c2ccncc2)cc(OC)c1', 'Cc1ccccc1COC(=O)c1[nH+]ccn1C', 'CO[C@@H]1CNc2ncnc([O-])c2C1=O', 'O=C(OCc1ccc([N+](=O)[O-])cc1)C1(c2ccc(F)cc2)C2CCC1CC2','C1=C[C@@]23N=CC=N[C@@]2(C=C1)N(C[C@@H](c1ccccc1)c1ncc[nH]1)N=N3', 'C[C@@]1(C(O)(O)Cc2ccoc2)CC(=O)CN1c1ccccc1C(F)F', 'O=C1C[C@@]2(C=CCC2)Nc2cccc(NC(=O)[C@@H]3C[C@H](N4CCCC4)c4ccccc4O3)c21', 'C[C@@H](Cc1c[nH]cc1N)OC(=O)C(=O)N1C[C@@H]2C=CC=C[C@@H]2C[C@@H](C)C1', 'CS[C@]1(c2ccccc2Oc2ccccc2Cl)CCN(C(=O)[C@H]2C[C@H]2[NH3+])C1', 'C[C@@](N)(c1ccc(I)cc1)c1ncc2cc(N)ccc2n1', 'CN(S)c1cccnc1CCC(=O)N1CC[NH+](C)CC1', 'C[C@]12C=CC=C[C@]1(O)NC=C2NC(=O)Cc1ccccc1O', 'CC1=CNN2CN=C(C(F)(F)F)C(NCc3cccc(Br)c3)=C12', 'Cc1nc(C2=NN[C@@]3(C([NH3+])([NH3+])c4cccnc4)C(Cl)=CC=CN23)no1', 'C#C[C@]1(Nc2c(C)cccc2C(F)F)NC(=O)CC[C@H]1N', 'N[C@]1(c2ccc(NC(=O)C3CCCC3)cc2)N[C@@]23C=CC=C[C@@]2(NC=N3)N1', 'Cc1cnc(C(=O)NN)nc1', 'Cc1ccc2c(c1C)S[C@H]([NH+]1CNC=C(CO[C@H]3CCCO3)C1)N2', 'O=C1C[C@@H](CCNC(=O)c2cccc(Br)c2)c2ccccc2N1', 'N[C@H]1CNCC(=O)N1C(=O)Oc1cccc2ccccc12', 'N[C@@]1(C(=O)NCC2(c3ccccc3)CC2)NC=CC=C1S(=O)(=O)N1CCCC1', 'C[C@H](Sc1cnn([C@@H](C)[NH3+])c1)c1ccc(C=O)cc1']

In [23]:

scores=0
for molec in randmols: #topmolecules
    queryfp = rdkit.Chem.RDKFingerprint(rdkit.Chem.MolFromSmiles(molec))
    scores = rdkit.Chem.DataStructs.BulkTanimotoSimilarity(queryfp,fps)
    print(molec)
    print("Most sim: "+targetaffins["SMILES"][np.argmax(scores)]+" uniprot: "+targetaffins["uniprots"][np.argmax(scores)])
    print(max(scores))

Nc1nnn(-c2ccccc2F)c1CS[C@]12C=CC=C[C@@H]1SC=N2
Most sim: COCC1=C(N2C(=C1C3=CC(=C(C=C3)NC(=O)NC4=C(C=CC(=C4)C(F)(F)F)F)F)C(=NC=N2)N)CN5CCOCC5 uniprot: P36888
0.5342841470104224
COC(=O)Cc1ccc(NS(C)(=O)=O)cc1
Most sim: CS(=O)(=O)NC1=CC=CC(=C1)CC(=O)NC2=NC(=CS2)C3=CNC4=C3C=CC=N4 uniprot: P36888
0.2447418738049713
Cc1cc(F)cc(C([NH3+])([NH3+])CN(C)C)c1
Most sim: C[C@H](C1=CC(=CC(=C1)F)F)NC2=CC3=C(NN=C3C=C2)/C=C/C4=CC=CC=N4 uniprot: P36888
0.2741228070175439
CC1(NC(=O)c2cnn(-c3ccccc3)n2)CCCCC1
Most sim: C1=CC(=CC=C1NC(=O)C2=NN(C=CC2=O)C3=CC(=C(C=C3)Cl)C(F)(F)F)OC4=C5C=CNC5=NC=C4 uniprot: P36888
0.3149374540103017
O=C[C@H](NC(=O)c1cccc2cccnc12)Nc1ccccc1
Most sim: C1=CC(=CC(=C1)N)C2=CC(=C3C(=C2)C=CC=N3)C(=O)N uniprot: P36888
0.512735326688815
Cc1c(N)cnn1C
Most sim: CC1=NN(C=C1NC(=O)CC2=CC=C(C=C2)OC3=C4C=C(C(=CC4=NC=C3)OC)OC)C uniprot: P36888
0.1853496115427303
Cc1cc(N)ccc1F
Most sim: C1=CC=C(C(=C1)C(=O)O)NC2=NC=NC(=N2)NC3=CC=C(C=C3)F uniprot: P36888
0.23214285714285715
CN(CCN1C(=O)c2ccccc2C1=O)